**Soil MCDA** for Phosphorus, Nitrogen, Potassium, Calcium, Magnesium, Sulphur and Zinc

In [ ]:
# Import modules
import ee
import geemap
import geopandas as gpd

In [11]:
# Initialize Earth Engine
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()

In [ ]:
# def get_vis_params(image, aoi):
#     # reduceRegion to get the minimum nutrient_extractable in the AOI
#     min_nutrient_extractable = image.reduceRegion(
#         reducer=ee.Reducer.min(),
#         geometry=aoi,
#         scale=3000,
#         maxPixels=9e8
#     )

#     # reduceRegion to get the maximum nutrient_extractable in the AOI
#     max_nutrient_extractable = image.reduceRegion(
#     reducer=ee.Reducer.max(),
#     geometry=aoi,
#     scale=3000,
#     maxPixels=9e8
#     )

#     # Get the result and print the minimum value
#     try:
#         nutrient_extractable_min_max = [min_nutrient_extractable.getInfo()['mean_0_20'], 
#                             max_nutrient_extractable.getInfo()['mean_0_20']]
#         vis_params = {'palette': ['darkred', 'red', 
#                           'white', 'lightblue', 'blue', 'darkblue'],
#                'min': nutrient_extractable_min_max[0], 'max': nutrient_extractable_min_max[1]}
        
#     except KeyError:
#         nutrient_extractable_min_max = [min_nutrient_extractable.getInfo()['constant'], 
#                             max_nutrient_extractable.getInfo()['constant']]
#         vis_params = {'palette': ['darkred', 'yellow', 'green', 'darkgreen'],
#                 'min': nutrient_extractable_min_max[0], 'max': nutrient_extractable_min_max[1]}
    
#     return vis_params

In [13]:
def selected_nutrient_suitability_range(nutrient):
    nutrient = nutrient.lower()
    
    #All units are in ppm
    nutrient_ranges = {
        'phosphorus': {
                'Soybean':    [(15, None, 3), (10, 15, 2), (5, 10, 1)],
                'Groundnut':  [(15, None, 3), (10, 15, 2), (5, 10, 1)],
                'Sugarbean':  [(15, None, 3), (10, 15, 2), (5, 10, 1)],
                'Navybean':   [(15, None, 3), (10, 15, 2), (5, 10, 1)],
                'Maize':      [(15, None, 3), (10, 15, 2), (5, 10, 1)],
                'Cowpea':     [(15, None, 3), (10, 15, 2), (5, 10, 1)],
                'Sunflower':  [(15, None, 3), (10, 15, 2), (5, 10, 1)],
            },
        'nitrogen': {
                'Soybean':    [(150, None, 3), (100, 150, 2), (50, 100, 1)],
                'Groundnut':  [(150, None, 3), (100, 150, 2), (50, 100, 1)],
                'Sugarbean':  [(150, None, 3), (100, 150, 2), (50, 100, 1)],
                'Navybean':   [(150, None, 3), (100, 150, 2), (50, 100, 1)],
                'Maize':      [(150, None, 3), (100, 150, 2), (50, 100, 1)],
                'Cowpea':     [(150, None, 3), (100, 150, 2), (50, 100, 1)],
                'Sunflower':  [(150, None, 3), (100, 150, 2), (50, 100, 1)],
            },
        'potassium': {
                'Soybean':    [(1170, None, 3), (780, 1170, 2), (390, 780, 1)],
                'Groundnut':  [(1170, None, 3), (780, 1170, 2), (390, 780, 1)],
                'Sugarbean':  [(1170, None, 3), (780, 1170, 2), (390, 780, 1)],
                'Navybean':   [(1170, None, 3), (780, 1170, 2), (390, 780, 1)],
                'Maize':      [(1170, None, 3), (780, 1170, 2), (390, 780, 1)],
                'Cowpea':     [(1170, None, 3), (780, 1170, 2), (390, 780, 1)],
                'Sunflower':  [(1170, None, 3), (780, 1170, 2), (390, 780, 1)],
            },
        'calcium': {
                'Soybean':    [(600, None, 3), (400, 600, 2), (200, 400, 1)],
                'Groundnut':  [(600, None, 3), (400, 600, 2), (200, 400, 1)],
                'Sugarbean':  [(600, None, 3), (400, 600, 2), (200, 400, 1)],
                'Navybean':   [(600, None, 3), (400, 600, 2), (200, 400, 1)],
                'Maize':      [(600, None, 3), (400, 600, 2), (200, 400, 1)],
                'Cowpea':     [(600, None, 3), (400, 600, 2), (200, 400, 1)],
                'Sunflower':  [(600, None, 3), (400, 600, 2), (200, 400, 1)],
            },
        'magnesium': {
                'Soybean':    [(600, None, 3), (360, 600, 2), (120, 360, 1)],
                'Groundnut':  [(600, None, 3), (360, 600, 2), (120, 360, 1)],
                'Sugarbean':  [(600, None, 3), (360, 600, 2), (120, 360, 1)],
                'Navybean':   [(600, None, 3), (360, 600, 2), (120, 360, 1)],
                'Maize':      [(600, None, 3), (360, 600, 2), (120, 360, 1)],
                'Cowpea':     [(600, None, 3), (360, 600, 2), (120, 360, 1)],
                'Sunflower':  [(600, None, 3), (360, 600, 2), (120, 360, 1)],
            },
        'sulphur': {
                'Soybean':    [(10, None, 3), (6, 10, 2), (3, 6, 1)],
                'Groundnut':  [(10, None, 3), (6, 10, 2), (3, 6, 1)],
                'Sugarbean':  [(10, None, 3), (6, 10, 2), (3, 6, 1)],
                'Navybean':   [(10, None, 3), (6, 10, 2), (3, 6, 1)],
                'Maize':      [(10, None, 3), (6, 10, 2), (3, 6, 1)],
                'Cowpea':     [(10, None, 3), (6, 10, 2), (3, 6, 1)],
                'Sunflower':  [(10, None, 3), (6, 10, 2), (3, 6, 1)],
            },
        'zinc': {
                'Soybean':    [(0.5, None, 3), (0.3, 0.5, 2), (0.1, 0.3, 1)],
                'Groundnut':  [(0.5, None, 3), (0.3, 0.5, 2), (0.1, 0.3, 1)],
                'Sugarbean':  [(0.5, None, 3), (0.3, 0.5, 2), (0.1, 0.3, 1)],
                'Navybean':   [(0.5, None, 3), (0.3, 0.5, 2), (0.1, 0.3, 1)],
                'Maize':      [(0.5, None, 3), (0.3, 0.5, 2), (0.1, 0.3, 1)],
                'Cowpea':     [(0.5, None, 3), (0.3, 0.5, 2), (0.1, 0.3, 1)],
                'Sunflower':  [(0.5, None, 3), (0.3, 0.5, 2), (0.1, 0.3, 1)],
            },

    }

    selected_nutrient_range = nutrient_ranges[nutrient]

    return selected_nutrient_range

In [ ]:
def get_nutrient_image(nutrient, aoi_ee):
    # Import nutrient_extractable dataset from GEE
    raw_phosphorus_extractable_dataset = ee.Image("ISDASOIL/Africa/v1/phosphorus_extractable") \
                .select('mean_0_20').clip(aoi_ee)
    raw_nitrogen_total_dataset = ee.Image("ISDASOIL/Africa/v1/nitrogen_total") \
                .select('mean_0_20').clip(aoi_ee)
    raw_potassium_extractable_dataset = ee.Image("ISDASOIL/Africa/v1/potassium_extractable") \
                .select('mean_0_20').clip(aoi_ee)
    raw_calcium_extractable_dataset = ee.Image("ISDASOIL/Africa/v1/calcium_extractable") \
                .select('mean_0_20').clip(aoi_ee)
    raw_magnesium_extractable_dataset = ee.Image("ISDASOIL/Africa/v1/magnesium_extractable") \
                .select('mean_0_20').clip(aoi_ee)
    raw_sulphur_extractable_dataset = ee.Image("ISDASOIL/Africa/v1/sulphur_extractable") \
                .select('mean_0_20').clip(aoi_ee)
    raw_zinc_extractable_dataset = ee.Image("ISDASOIL/Africa/v1/zinc_extractable") \
                .select('mean_0_20').clip(aoi_ee)
    
    nutrient = nutrient.lower()
    
    selected_nutrient = {
        'phosphorus': raw_phosphorus_extractable_dataset,
        'nitrogen': raw_nitrogen_total_dataset, 
        'potassium': raw_potassium_extractable_dataset, 
        'calcium': raw_calcium_extractable_dataset,
        'magnesium': raw_magnesium_extractable_dataset,
        'sulphur': raw_sulphur_extractable_dataset,
        'zinc': raw_zinc_extractable_dataset
    }

    nutrient_extractable_dataset = selected_nutrient[nutrient].divide(10).exp().subtract(1)

    return nutrient_extractable_dataset


In [15]:
# Get ranked nutrient_extractable suitability
def get_nutrient_suitability_image(crop, nutrient, aoi_ee):
    nutrient_image = get_nutrient_image(nutrient, aoi_ee)
    suitability_ranges = selected_nutrient_suitability_range(nutrient)

    # Standardize crop name input
    crop = crop.strip().replace(" ", "").title()
    ranges = suitability_ranges[crop]

    # Default suitability of 0 (no suitability)
    suitability_image = ee.Image(0).clip(aoi_ee)

    # Apply suitability levels
    for (min_val, max_val, score) in ranges:
        if max_val is None:
            mask = nutrient_image.gte(min_val)
        else:
            mask = nutrient_image.gte(min_val).And(nutrient_image.lt(max_val)) 

        suitability_image = suitability_image.where(mask, score)

    return suitability_image

In [ ]:
# # Open source table as a ee
# # country_gpkg = r"data\zambia_boundary.gpkg"
# country_gpkg = r"C:\Users\jedid\Documents\Code\github\crop_soil_mcda\data\zambia_boundary.gpkg"
# country_gdf = gpd.read_file(country_gpkg)
# country_ee = geemap.gdf_to_ee(country_gdf)

# # Preview the data
# # country_gdf.head()

In [ ]:
# nutrient = 'magnesium'
# crop = 'Soybean'

# soil_nutrient_image = get_nutrient_image(nutrient, country_ee)
# soil_nutrient_suitability_image = get_nutrient_suitability_image(crop, nutrient, country_ee)

In [ ]:
# # phosphorus_extractable visualisastion parameters
# soil_nutrient_vis_params = get_vis_params(soil_nutrient_image, country_ee)
# soil_nutrient_suitability_vis_params = get_vis_params(soil_nutrient_suitability_image, country_ee)

In [ ]:
# # Visualise layers
# m = geemap.Map()
# m.zoom_to_gdf(country_gdf)

# m.add_layer(country_ee, name= 'Zambia')

# m.add_ee_layer(soil_nutrient_image, vis_params= soil_nutrient_vis_params, name= f"{nutrient}_extractable")
# m.add_colorbar(soil_nutrient_vis_params, orientation='vertical',  
#                label= "phosphorus_extractable", layer_name= f"phosphorus_extractable")

# m.add_ee_layer(soil_nutrient_suitability_image, vis_params= soil_nutrient_suitability_vis_params, name= f"{crop} Suitability ({nutrient})")
# # m.add_colorbar(ranked_phosphorus_extractable_vis_params, orientation='vertical',  
#             #    label= "phosphorus_extractable Suitability", layer_name= f"{crop} phosphorus_extractable Suitability")
# m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…